In [ ]:
"""
Notebook to pose unposed cameras. Run after running 2-extrinsics.py

Pseudocode:
- Load cameras
    - If camera is posed add to posed_cameras
    - If camera is unposed add to unposed_cameras
- For each unposed camera in unposed_cameras do:
    - Find the closest posed camera
    - Select keypoints in overlapping views
    - Solve PnP to get pose of unposed camera
    - Add the posed camera to posed_cameras
    - Remove the unposed camera from unposed_cameras
    - Update extrinsics
"""





In [ ]:
# Load Cameras

import os
import sys
import numpy as np
import json

sys.path.append('..')
from configs.arguments import get_config_dict
from utils.multiview_utils import Camera, Calibration, MultiviewVids
from utils.io_utils import write_json, load_json
from utils.metadata_utils import get_cam_names
from utils.coordinate_utils import update_reconstruction, point_in_polygon, project_to_ground_plane_cv2
from utils.plot_utils import rotation_matrix, perp

from scipy.spatial.transform import Rotation as R
from skspatial.objects import Point, Vector, Plane, Points, Line
import pyransac3d as pyrsc

import copy
import cv2
import ipywidgets as widgets
import ipympl
import matplotlib.pyplot as plt
%matplotlib widget

In [ ]:


arv_copy = sys.argv
sys.argv = ['pop']
sys.argv.append('-cfg')
config_path = os.path.abspath('../../project_config.yaml')
sys.argv.append(config_path)
sys.argv.append('-dr')
root_path = os.path.abspath('../../data/')
sys.argv.append(root_path)
sys.argv.append('-l')
sys.argv.append('info')

# load arguments from the arg parser
config = get_config_dict()
data_root = config["main"]["data_root"]
calib_dir = os.path.join(data_root, '0-calibration', 'calibs')
video_dir = os.path.join(data_root, 'raw_data', 'footage')
reconstruction_dir = os.path.join(data_root, '0-calibration', 'opensfm', 'undistorted', 'reconstruction.json')
omni_tag = '360'


reconstruction_dir = os.path.join(data_root, '0-calibration', 'opensfm', 'reconstruction.json')
if not os.path.exists(reconstruction_dir):
    print( "Reconstruction not found. Please run 0-calibration/2-extrinsics.py first.")
    

In [20]:
import pickle
import numpy

with open('/Users/grosche/Documents/GitHub/CVLAB/MARMOT/data/cam1_432.jpg_matches.pkl', 'rb') as f:
    matches = pickle.load(f)

features = numpy.load('/Users/grosche/Documents/GitHub/CVLAB/MARMOT/data/cam1_432.jpg.features.npz')
print(features['points'])

[[-1.04274049e-01 -9.06276703e-02  9.35720222e-04  2.11207199e+01]
 [-1.33819669e-01  1.42704649e-02  9.35757125e-04  3.27680481e+02]
 [ 7.66289383e-02 -1.05478048e-01  9.35885240e-04  2.83030731e+02]
 ...
 [ 1.23492815e-01  5.91947548e-02  5.46103157e-02  3.29975861e+02]
 [-3.10146511e-01 -6.52066097e-02  8.98958743e-02  1.80739166e+02]
 [-1.20714344e-01 -6.97865039e-02  1.15327813e-01  1.31264725e+02]]


In [5]:
matches

{'cam4_12030.jpg': array([], dtype=float64),
 'cam360_765.jpg': array([], dtype=float64),
 'cam360_587.jpg': array([], dtype=float64),
 'cam3_12160.jpg': array([], dtype=float64),
 'cam3_1351.jpg': array([], dtype=float64),
 'cam360_232.jpg': array([[ 2844, 11180],
        [ 4954, 16416],
        [ 5210, 18810],
        [ 4582, 13102],
        [ 4896, 15595],
        [ 5087, 17286],
        [ 1492,  9989],
        [ 3002, 13082],
        [ 5199, 18771],
        [ 2470,  6521],
        [ 5091, 18374],
        [ 5059, 18467],
        [ 4793, 16453],
        [ 2831,  5776],
        [ 5142, 18336],
        [ 4089, 14505],
        [ 4588, 15211],
        [  919,   249],
        [ 4334, 15361],
        [ 2688,  3834],
        [ 2375, 11971],
        [ 2329,  5279],
        [ 5104, 17504],
        [ 2361,  9459],
        [  759,  2243],
        [ 1651, 14289],
        [ 4155, 15961],
        [ 4407, 10882],
        [ 4895, 15596],
        [  764,  2929],
        [ 1391,  9107],
        [  632